In [1]:
from tensorzinb.tensorzinb import TensorZINB
import numpy as np
import time

from scipy.stats import uniform, binom, nbinom, bernoulli
import statsmodels.api as sm

## Poisson initialization validation for Negative Binomial

In [2]:
np.random.seed(1)                 # set seed to replicate example
nobs= 25000                          # number of obs in model 

xb = np.ones((nobs,1)) *0.7
theta = 0.5

exb = np.exp(xb)
nby = nbinom.rvs(exb, theta)
X=np.ones((nobs,1))

In [3]:
zinbo= TensorZINB(nby.reshape((-1,1)),X)

In [4]:
inits = zinbo._poisson_init()
inits

{'x_mu': array([[0.69494556]]), 'theta': array([[0.69711051]])}

In [5]:
r=zinbo.fit()
r

Metal device set to: Apple M2


{'llf_total': -47019.82560537191,
 'llfs': array([-47019.82560537]),
 'aic_total': 94043.65121074382,
 'aics': array([94043.65121074]),
 'df_model_total': 2,
 'df': 2,
 'weights': {'x_mu': array([[0.6951688]], dtype=float32),
  'theta': array([[0.69479316]], dtype=float32)},
 'cpu_time': 0.49103713035583496,
 'num_sample': 25000,
 'epochs': 51}

## Poisson initialization validation for Zero-inflated Negative Binomial 

In [6]:
np.random.seed(1)                    # set seed to replicate example
nobs= 25000                          # number of obs in model 

xb = np.ones((nobs,1)) *0.7
theta = 0.5

xc = 2.0
exc = 1.0 / (1.0 + np.exp(-xc))

p = bernoulli.rvs(exc, size=(nobs,1))

exb = np.exp(xb)
nby = nbinom.rvs(exb, theta)*p
X=np.ones((nobs,1))

In [7]:
zinbo= TensorZINB(nby.reshape((-1,1)),X,exog_infl=X)

In [8]:
inits = zinbo._poisson_init()
inits

{'x_mu': array([[0.57050486]]),
 'x_pi': array([[-3.52493127]]),
 'theta': array([[0.33882783]])}

In [9]:
r=zinbo.fit()
r

{'llf_total': -45113.60175795488,
 'llfs': array([-45113.60175795]),
 'aic_total': 90233.20351590976,
 'aics': array([90233.20351591]),
 'df_model_total': 3,
 'df': 3,
 'weights': {'x_mu': array([[0.6884514]], dtype=float32),
  'x_pi': array([[-2.072716]], dtype=float32),
  'theta': array([[0.6567552]], dtype=float32)},
 'cpu_time': 2.100802183151245,
 'num_sample': 25000,
 'epochs': 1133}

## Negative Binomial

In [10]:
np.random.seed(1)                 # set seed to replicate example
nobs= 25000                          # number of obs in model 

x1 = binom.rvs(1, 0.6, size=nobs)   # categorical explanatory variable
x2 = uniform.rvs(size=nobs)         # real explanatory variable

theta = 0.5
X = sm.add_constant(np.column_stack((x1, x2)))
beta = [1.0, 2.0, -1.5]
xb = np.dot(X, beta)          # linear predictor

exb = np.exp(xb)
nby = nbinom.rvs(exb, theta)

In [11]:
zinbo= TensorZINB(nby.reshape((-1,1)),X)

In [12]:
r=zinbo.fit()
r

{'llf_total': -59241.24790102232,
 'llfs': array([-59241.24790102]),
 'aic_total': 118490.49580204464,
 'aics': array([118490.49580204]),
 'df_model_total': 4,
 'df': 4,
 'weights': {'x_mu': array([[ 1.0025651],
         [ 1.9950333],
         [-1.4982768]], dtype=float32),
  'theta': array([[2.1333747]], dtype=float32)},
 'cpu_time': 0.48426389694213867,
 'num_sample': 25000,
 'epochs': 260}

this is the definition of dispersion in statsmodels

In [13]:
np.exp(-r['weights']['theta'])

array([[0.11843693]], dtype=float32)

In [14]:
start_time = time.time()
mod = sm.NegativeBinomial(nby, X).fit(maxiter=100, disp=False, warn_convergence=False)
cpu_time = time.time() - start_time
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     NegativeBinomial Regression Results                      
==============================================================================
Dep. Variable:                      y   No. Observations:                25000
Model:               NegativeBinomial   Df Residuals:                    24997
Method:                           MLE   Df Model:                            2
Date:                Tue, 16 May 2023   Pseudo R-squ.:                  0.2064
Time:                        23:45:16   Log-Likelihood:                -59241.
converged:                       True   LL-Null:                       -74651.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0028      0.010     96.502      0.000       0.982       1.023
x1             1.9953      0.010    200.495      0.000       1.976       2.015
x2            -1.4980      0.013   -118.673      0.000      -1.523      -1.473
alpha          0.1184      0.003     42.072      0.000       0.113       0.124
==============================================================================
"""

In [15]:
cpu_time

0.10850691795349121

TensorZINB matches statsmodels for Negative Binomial

## Zero-inflated Negative Binomial 

In [16]:
np.random.seed(1)                 # set seed to replicate example
nobs= 25000                          # number of obs in model 

x1 = binom.rvs(1, 0.6, size=nobs)   # categorical explanatory variable
x2 = uniform.rvs(size=nobs)         # real explanatory variable

theta = 0.5
X = sm.add_constant(np.column_stack((x1, x2)))
beta = [1.0, 2.0, -1.5]
xb = np.dot(X, beta)          # linear predictor

exb = np.exp(xb)

xc = 3.0
exc = 1.0 / (1.0 + np.exp(-xc))

p = bernoulli.rvs(exc, size=(nobs,1))

nby = nbinom.rvs(exb, theta).reshape((-1,1))*p
X_infl=np.ones((nobs,1))

In [17]:
zinbo= TensorZINB(nby.reshape((-1,1)),X,exog_infl=X_infl)

In [18]:
r=zinbo.fit()
r

{'llf_total': -59281.92643573915,
 'llfs': array([-59281.92643574]),
 'aic_total': 118573.8528714783,
 'aics': array([118573.85287148]),
 'df_model_total': 5,
 'df': 5,
 'weights': {'x_mu': array([[ 1.0397824],
         [ 1.9678782],
         [-1.5097415]], dtype=float32),
  'x_pi': array([[-2.7315001]], dtype=float32),
  'theta': array([[2.196404]], dtype=float32)},
 'cpu_time': 0.6659359931945801,
 'num_sample': 25000,
 'epochs': 309}

this is the definition of dispersion in statsmodels

In [19]:
np.exp(-r['weights']['theta'])

array([[0.11120233]], dtype=float32)

statsmodels cannot solve

In [20]:
start_time = time.time()
mod = sm.ZeroInflatedNegativeBinomialP(nby.reshape((-1,1)),X,exog_infl=X_infl).fit()
cpu_time = time.time() - start_time
mod.summary()

         Current function value: nan
         Iterations: 1
         Function evaluations: 112
         Gradient evaluations: 112


<class 'statsmodels.iolib.summary.Summary'>
"""
                     ZeroInflatedNegativeBinomialP Regression Results                    
=========================================================================================
Dep. Variable:                                 y   No. Observations:                25000
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                    24997
Method:                                      MLE   Df Model:                            2
Date:                           Tue, 16 May 2023   Pseudo R-squ.:                     nan
Time:                                   23:45:21   Log-Likelihood:                    nan
converged:                                 False   LL-Null:                       -72991.
Covariance Type:                       nonrobust   LLR p-value:                       nan
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
inflate_const  -351.6158        nan        nan        nan         nan         nan
const           213.8910        nan        nan        nan         nan         nan
x1              100.9036        nan        nan        nan         nan         nan
x2              101.6834        nan        nan        nan         nan         nan
alpha          -339.1602        nan        nan        nan         nan         nan
=================================================================================
"""

statsmodels cannot solve even with Poisson init

In [21]:
inits = zinbo._poisson_init()
start_params=np.concatenate([inits['x_pi'].flatten(),inits['x_mu'].flatten(),np.exp(-inits['theta'].flatten())])

In [22]:
start_time = time.time()
mod = sm.ZeroInflatedNegativeBinomialP(nby.reshape((-1,1)),X,exog_infl=X_infl).fit(start_params=start_params)
cpu_time = time.time() - start_time
mod.summary()

         Current function value: nan
         Iterations: 2
         Function evaluations: 113
         Gradient evaluations: 113


<class 'statsmodels.iolib.summary.Summary'>
"""
                     ZeroInflatedNegativeBinomialP Regression Results                    
=========================================================================================
Dep. Variable:                                 y   No. Observations:                25000
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                    24997
Method:                                      MLE   Df Model:                            2
Date:                           Tue, 16 May 2023   Pseudo R-squ.:                     nan
Time:                                   23:45:24   Log-Likelihood:                    nan
converged:                                 False   LL-Null:                       -72991.
Covariance Type:                       nonrobust   LLR p-value:                       nan
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
inflate_const     7.1325        nan        nan        nan         nan         nan
const           -78.1776        nan        nan        nan         nan         nan
x1              -88.9186        nan        nan        nan         nan         nan
x2              -47.4675        nan        nan        nan         nan         nan
alpha          -679.3743        nan        nan        nan         nan         nan
=================================================================================
"""

statsmodels only works when given optimized params generated by TensorZINB. It still takes very long time in this case.

In [23]:
inits = r['weights']
start_params=np.concatenate([inits['x_pi'].flatten(),inits['x_mu'].flatten(),np.exp(-inits['theta'].flatten())])

In [24]:
start_time = time.time()
mod = sm.ZeroInflatedNegativeBinomialP(nby.reshape((-1,1)),X,exog_infl=X_infl).fit(start_params=start_params)
cpu_time = time.time() - start_time
mod.summary()

Optimization terminated successfully.
         Current function value: 2.371275
         Iterations: 5
         Function evaluations: 8
         Gradient evaluations: 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                     ZeroInflatedNegativeBinomialP Regression Results                    
=========================================================================================
Dep. Variable:                                 y   No. Observations:                25000
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                    24997
Method:                                      MLE   Df Model:                            2
Date:                           Tue, 16 May 2023   Pseudo R-squ.:                  0.1878
Time:                                   23:45:25   Log-Likelihood:                -59282.
converged:                                  True   LL-Null:                       -72991.
Covariance Type:                       nonrobust   LLR p-value:                     0.000
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
inflate_const    -2.7314      0.038    -72.583      0.000      -2.805      -2.658
const             1.0393      0.011     91.869      0.000       1.017       1.061
x1                1.9675      0.011    181.047      0.000       1.946       1.989
x2               -1.5102      0.013   -116.512      0.000      -1.536      -1.485
alpha             0.1112      0.003     39.645      0.000       0.106       0.117
=================================================================================
"""

In [25]:
cpu_time

0.3456001281738281

TensorZINB matches statsmodels for Zero-inflated Negative Binomial when TensorZINB results are used for statsmodels initialization. The estimated weights from TensorZINB are close to true values used to generate the samples.